In [17]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
from markovBike.forecast.forecast import calculate_daily_rides, get_daily_rides_with_weather, forecast_number_users
from markovBike.data_source.source import get_stations_data, database_queries
from markovBike.data_source.preprocess import preprocess_trips_data

import pandas as pd
import os

In [19]:
n_trips = 60

trips_raw = get_stations_data(database_queries(n_trips)['trips'], verbose=True)

trips_raw['date'] = pd.to_datetime(trips_raw['starttime']).dt.date

daily_rides = calculate_daily_rides(trips_raw)

merged_df = trips_raw.merge(daily_rides, on='date')

weather_dataframe = get_daily_rides_with_weather(merged_df, os.environ.get('API_KEY'))

Bike station table with shape (60, 15). Columns are: 

tripduration                             Int64
starttime                  datetime64[ns, UTC]
stoptime                   datetime64[ns, UTC]
start_station_id                         Int64
start_station_name                      object
start_station_latitude                 float64
start_station_longitude                float64
end_station_id                           Int64
end_station_name                        object
end_station_latitude                   float64
end_station_longitude                  float64
bikeid                                   Int64
usertype                                object
birth_year                               Int64
gender                                  object
dtype: object



ConnectionError: HTTPSConnectionPool(host='api.openweathermap.org', port=443): Max retries exceeded with url: /data/2.5/weather?q=New%20York&units=imperial&appid=3eb15a4d67cbf82c0c9560768a851403&dt=2014-06-06 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x14b022320>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))

In [20]:
weather_dataframe.head(3)

,tripduration,starttime,stoptime,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,...,bikeid,usertype,birth_year,gender,date,daily_rides,temperature,description,humidity,wind_speed
0,107,2014-06-06 13:02:01+00:00,2014-06-06 13:03:48+00:00,520,W 52 St & 5 Ave,40.759923,-73.976485,520,W 52 St & 5 Ave,40.759923,...,16692,Subscriber,1977,male,2014-06-06,1,38.75,clear sky,54,10.36
1,1105,2013-08-24 17:03:52+00:00,2013-08-24 17:22:17+00:00,520,W 52 St & 5 Ave,40.759923,-73.976485,520,W 52 St & 5 Ave,40.759923,...,15352,Customer,<NA>,unknown,2013-08-24,1,38.84,clear sky,54,10.36
2,990,2014-04-25 11:52:28+00:00,2014-04-25 12:08:58+00:00,520,W 52 St & 5 Ave,40.759923,-73.976485,520,W 52 St & 5 Ave,40.759923,...,20194,Subscriber,1973,male,2014-04-25,1,38.84,clear sky,54,10.36


In [ ]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

numerical_features = weather_dataframe.select_dtypes(
    include=numerics).columns.drop(
        ['start_station_id', 'end_station_id', 'bikeid'])

categorical_features = weather_dataframe.select_dtypes(
    include='object').columns.drop([
        'start_station_name', 'end_station_name'
    ]).append(pd.Index(['start_station_id', 'end_station_id', 'bikeid']))

boolean_features = weather_dataframe.select_dtypes(include='bool').columns

In [ ]:
numerical_features

Index(['tripduration', 'start_station_latitude', 'start_station_longitude',
       'end_station_latitude', 'end_station_longitude', 'birth_year',
       'daily_rides', 'temperature', 'humidity', 'wind_speed'],
      dtype='object')

In [ ]:
weather_dataframe_preproc = preprocess_trips_data(
    weather_dataframe,
    index=None,
    numerical_features=numerical_features,
    categorical_features=categorical_features,
    boolean_features=boolean_features,
    drops=None,
    verbose=True)


 📶 Columns ['tripduration', 'starttime', 'stoptime', 'start_station_id', 'start_station_name', 'start_station_latitude', 'start_station_longitude', 'end_station_id', 'end_station_name', 'end_station_latitude', 'end_station_longitude', 'bikeid', 'usertype', 'birth_year', 'gender', 'date', 'daily_rides', 'temperature', 'description', 'humidity', 'wind_speed']

 📶 Numericals ['tripduration', 'start_station_latitude', 'start_station_longitude', 'end_station_latitude', 'end_station_longitude', 'birth_year', 'daily_rides', 'temperature', 'humidity', 'wind_speed']

 📶 Categoricals ['usertype', 'gender', 'date', 'description', 'start_station_id', 'end_station_id', 'bikeid']

 📶 Booleans []
Index(['tripduration', 'starttime', 'stoptime', 'start_station_id',
       'start_station_name', 'start_station_latitude',
       'start_station_longitude', 'end_station_id', 'end_station_name',
       'end_station_latitude', 'end_station_longitude', 'bikeid', 'usertype',
       'birth_year', 'gender', 'dat

ValueError: A given column is not a column of the dataframe

In [ ]:
forecast_number_users(
    weather_dataframe_preproc,
    X=['temperature', 'humidity', 'wind_speed', 'description'],
    y='daily_rides',
    test_size=0.2,
    lstm_units=64,
    lstm_epochs=50)